# План выполнения проекта:
### - Загрузка и подготовка данных.
### - Построение и обучение модели.
### - Расчет прибыли и рисков.¶
### - Вывод.

# 1. Загрузка и подготовка данных

In [1]:
import pandas as pd

In [2]:
geo_data_0 = pd.read_csv('/datasets/geo_data_0.csv')
geo_data_1 = pd.read_csv('/datasets/geo_data_1.csv')
geo_data_2 = pd.read_csv('/datasets/geo_data_2.csv')

In [1]:
geo_data_0

In [2]:
geo_data_1

In [3]:
geo_data_2

In [6]:
geo_data_0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [7]:
geo_data_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [8]:
geo_data_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


- Мы имеем три набора данных geo_data_0, geo_data_1, geo_data_2. В каждом из них по 5 колонок и по 100000 строк.

- Пропущенных значений нет.

- Тип данных изменять не надо.

Проверим нет ли дубликатов:

In [9]:
geo_data_0.duplicated().sum()

0

In [10]:
geo_data_1.duplicated().sum()

0

In [11]:
geo_data_2.duplicated().sum()

0

- Дубликатов нет

Проверим есть ли строки с одинаковым уникальным идентификатором скважины в столбце id:

In [12]:
geo_data_0['id'].value_counts().head(13)

bsk9y    2
QcMuo    2
AGS9W    2
HZww2    2
74z30    2
A5aEY    2
bxg6G    2
fiKDv    2
TtcGQ    2
Tdehs    2
sE1Li    1
Hq8XU    1
pRNTD    1
Name: id, dtype: int64

In [13]:
geo_data_1['id'].value_counts().head()

wt4Uk    2
5ltQ6    2
bfPNe    2
LHZR0    2
Bqrgq    1
Name: id, dtype: int64

In [14]:
geo_data_2['id'].value_counts().head()

Vcm5J    2
KUPhW    2
xCHr8    2
VF7Jo    2
Ju5bb    1
Name: id, dtype: int64

In [15]:
geo_data_0.query('id == "bxg6G"')

,id,f0,f1,f2,product
1364,bxg6G,0.411645,0.856830,-3.653440,73.604260
41724,bxg6G,-0.823752,0.546319,3.630479,93.007798


- Есть повторяющиеся значения в столбце 'id', при этом остальные признаки в этих строках различаются. Надо подумать что с ними делать. Можно их оставить, так как их мало.

Удалим столбец 'id':

In [16]:
geo_data_0 = geo_data_0.drop(['id'], axis=1)

In [17]:
geo_data_1 = geo_data_1.drop(['id'], axis=1)

In [18]:
geo_data_2 = geo_data_2.drop(['id'], axis=1)

# 2. Обучение и проверка модели

Обучим и проверим модель для каждого региона.

#### 2.1. Разобьем данные на обучающую и валидационную выборки в соотношении 75:25.

Используем **train_test_split**:

In [19]:
from sklearn.model_selection import train_test_split
geo_data_0_train, geo_data_0_valid = train_test_split(geo_data_0, test_size=0.25, random_state=12345)    

In [20]:
geo_data_1_train, geo_data_1_valid = train_test_split(geo_data_1, test_size=0.25, random_state=12345)        

In [21]:
geo_data_2_train, geo_data_2_valid = train_test_split(geo_data_2, test_size=0.25, random_state=12345)        

Получили наборы данных:

geo_data_0_train, geo_data_0_valid

geo_data_1_train, geo_data_1_valid

geo_data_2_train, geo_data_2_valid

Запишем в переменные **features_train_0** и **target_train_0** соответсвенно признаки и целевой признак из обучающей выборки geo_data_0_train.
В переменные **features_valid_0** и **target_valid_0** соответсвенно признаки и целевой признак из валидационной выборки geo_data_0_valid.

In [22]:
features_train_0 = geo_data_0_train.drop(['product'], axis=1)
target_train_0 = geo_data_0_train['product']
features_valid_0 = geo_data_0_valid.drop(['product'], axis=1)
target_valid_0 = geo_data_0_valid['product']

Сделаем тоже самое для набора данных geo_data_1 и geo_data_2.

In [23]:
features_train_1 = geo_data_1_train.drop(['product'], axis=1)
target_train_1 = geo_data_1_train['product']
features_valid_1 = geo_data_1_valid.drop(['product'], axis=1)
target_valid_1 = geo_data_1_valid['product']

In [24]:
features_train_2 = geo_data_2_train.drop(['product'], axis=1)
target_train_2 = geo_data_2_train['product']
features_valid_2 = geo_data_2_valid.drop(['product'], axis=1)
target_valid_2 = geo_data_2_valid['product']

Масштабируем признаки.
Приводим признаки к одному масштабу для этого используем стандартизацию данных.

Импортируем StandardScaler из библиотеки:

In [25]:
from sklearn.preprocessing import StandardScaler

Настраиваем на обучающих выборках для каждого региона:

In [26]:
scaler_0 = StandardScaler()
scaler_0.fit(features_train_0)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [27]:
scaler_1 = StandardScaler()
scaler_1.fit(features_train_1)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [28]:
scaler_2 = StandardScaler()
scaler_2.fit(features_train_2)

StandardScaler(copy=True, with_mean=True, with_std=True)

Преобразуем обучающие, валидационные выборки для каждого региона.

In [29]:
features_train_0 = scaler_0.transform(features_train_0)
features_valid_0 = scaler_0.transform(features_valid_0)

In [30]:
features_train_1 = scaler_1.transform(features_train_1)
features_valid_1 = scaler_1.transform(features_valid_1)

In [31]:
features_train_2 = scaler_2.transform(features_train_2)
features_valid_2 = scaler_2.transform(features_valid_2)

#### 2.2. Обучим модель и сделаем предсказания на валидационной выборке.

Из условия - "Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые)". Поэтому используем линейную регрессию.

In [32]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

Модель для региона geo_0:

In [33]:
model_0 = LinearRegression()

In [34]:
model_0.fit(features_train_0, target_train_0)
predicted_valid_0 = model_0.predict(features_valid_0)

Модель для региона geo_1:

In [35]:
model_1 = LinearRegression()
model_1.fit(features_train_1, target_train_1)
predicted_valid_1 = model_1.predict(features_valid_1)

Модель для региона geo_2:

In [36]:
model_2 = LinearRegression()
model_2.fit(features_train_2, target_train_2)
predicted_valid_2 = model_2.predict(features_valid_2)

#### 2.3. Сохраним предсказания и правильные ответы на валидационной выборке.

Таблица с предсказаниями и правильными ответами целевого признака  — объёма запасов в скважине "product" для региона geo_0:

In [37]:
prediction_and_target_0 = pd.DataFrame()
prediction_and_target_0['target_valid_0'] = target_valid_0
prediction_and_target_0['predicted_valid_0'] = predicted_valid_0
prediction_and_target_0

,target_valid_0,predicted_valid_0
71751,10.038645,95.894952
80493,114.551489,77.572583
2655,132.603635,77.892640
53233,169.072125,90.175134
91141,122.325180,70.510088
...,...,...
12581,170.116726,103.037104
18456,93.632175,85.403255
73035,127.352259,61.509833
63834,99.782700,118.180397


Таблица с предсказаниями и правильными ответами объёма запасов в скважине для региона geo_1:

In [38]:
prediction_and_target_1 = pd.DataFrame()
prediction_and_target_1['target_valid_1'] = target_valid_1
prediction_and_target_1['predicted_valid_1'] = predicted_valid_1
prediction_and_target_1

,target_valid_1,predicted_valid_1
71751,80.859783,82.663314
80493,53.906522,54.431786
2655,30.132364,29.748760
53233,53.906522,53.552133
91141,0.000000,1.243856
...,...,...
12581,137.945408,136.869211
18456,110.992147,110.693465
73035,137.945408,137.879341
63834,84.038886,83.761966


Таблица с предсказаниями и правильными ответами для региона geo_2:

In [39]:
prediction_and_target_2 = pd.DataFrame()
prediction_and_target_2['target_valid_2'] = target_valid_2
prediction_and_target_2['predicted_valid_2'] = predicted_valid_2
prediction_and_target_2

,target_valid_2,predicted_valid_2
71751,61.212375,93.599633
80493,41.850118,75.105159
2655,57.776581,90.066809
53233,100.053761,105.162375
91141,109.897122,115.303310
...,...,...
12581,28.492402,78.765887
18456,21.431303,95.603394
73035,125.487229,99.407281
63834,99.422903,77.779912


Для geo_0 и geo_2 имеется существенное различие между предсказаниями и правильными ответами объёма запасов сырья.

##### 2.4. Напечатаем на экране средний запас предсказанного сырья и RMSE модели.

Запишем в таблицу средний запас предсказанного сырья, реальный запас сырья, RMSE, MAE и R2:

In [40]:
data = pd.DataFrame(index = ['geo_0', 'geo_1', 'geo_2'])
data['RMSE'] = [mean_squared_error(target_valid_0, predicted_valid_0)**0.5, 
                mean_squared_error(target_valid_1, predicted_valid_1)**0.5, 
                mean_squared_error(target_valid_2, predicted_valid_2)**0.5]
data['R2'] = [r2_score(target_valid_0, predicted_valid_0), 
              r2_score(target_valid_1, predicted_valid_1), 
              r2_score(target_valid_2, predicted_valid_2)]
data['MAE'] = [mean_absolute_error(target_valid_0, predicted_valid_0), 
              mean_absolute_error(target_valid_1, predicted_valid_1), 
              mean_absolute_error(target_valid_2, predicted_valid_2)]
data['predicted_mean'] = [predicted_valid_0.mean(), 
                          predicted_valid_1.mean(), 
                          predicted_valid_2.mean()]
data['real_mean'] = [target_valid_0.mean(), 
                     target_valid_1.mean(), 
                     target_valid_2.mean()]
data

,RMSE,R2,MAE,predicted_mean,real_mean
geo_0,37.579422,0.279943,30.919601,92.592568,92.078597
geo_1,0.893099,0.999623,0.718766,68.728547,68.723136
geo_2,40.029709,0.205248,32.792652,94.965046,94.884233


#### 2.5. Проанализируем результаты.

Из полученных данных видим, что мы получили разные по качеству модели для каждого региона. Модель для региона geo_1 нам подходит, так как она имеет хорошие метрики качества. А модели для регионов geo_0 и geo_2 имеют большие RMSE и MAE и очень маленький коэффициент детермминации R2. Эти модели надо доработать. При этом средний запас предсказанного сырья  такой же как среднее занчения реального запаса для всех моделей.

# 3. Подготовка к расчёту прибыли

#### 3.1. Все ключевые значения для расчётов сохраним в отдельных переменных.

In [41]:
wells = 500
top_wells_qty = 200
budget = 10000000000
profit_per_1000bbl = 450000

#### 3.2. Рассчитаем достаточный объём сырья для безубыточной разработки новой скважины. Сравним полученный объём сырья со средним запасом в каждом регионе.

Достаточный объём сырья для безубыточной разработки новой скважины:

In [42]:
loss_free_product = round((budget / top_wells_qty) / profit_per_1000bbl, 2)
loss_free_product

111.11

Запишем в таблицу:

In [43]:
data['loss_free_product'] = loss_free_product
data

,RMSE,R2,MAE,predicted_mean,real_mean,loss_free_product
geo_0,37.579422,0.279943,30.919601,92.592568,92.078597,111.11
geo_1,0.893099,0.999623,0.718766,68.728547,68.723136,111.11
geo_2,40.029709,0.205248,32.792652,94.965046,94.884233,111.11


#### 3.3. Напишем выводы по этапу подготовки расчёта прибыли.

Достаточный объём сырья для безубыточной разработки новой скважины во всех регионах больше среднего запаса в регионе. Нужно отбирать лучшие скажины.

# 4. Расчёт прибыли и рисков 

#### Напишем функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели:

- 4.1. Выберем скважины с максимальными значениями предсказаний.
- 4.2. Просуммируем целевое значение объёма сырья, соответствующее этим предсказаниям.
- 4.3. Рассчитаем прибыль для полученного объёма сырья.

In [44]:
def revenue(target_valid, predicted_valid, count):
    predicted_sorted = predicted_valid.sort_values(ascending=False).head(count)
    selected_target_sum = target_valid[predicted_sorted.index].sum()
    revenue_sum = selected_target_sum * profit_per_1000bbl
    return revenue_sum

### 5. Посчитаем риски и прибыль для каждого региона:

##### 5.1. Примените технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли.

In [45]:
import numpy as np
def boot(target_valid, predicted_valid):
    state = np.random.RandomState(12345)
    values = []
    for i in range(1000):
        target_subsample = target_valid.sample(n=500, replace=True, random_state=state)
        predicted_subsample = predicted_valid[target_subsample.index]
        values.append(revenue(target_subsample, predicted_subsample, 200))
    values = pd.Series(values)
    return values

##### 5.2. Найдите среднюю прибыль, 95%-й доверительный интервал и риск убытков. Убыток — это отрицательная прибыль.

In [46]:
from scipy import stats as st

In [47]:
predicted_valid_0 = pd.Series(predicted_valid_0)
target_valid_0 = target_valid_0.reset_index(drop=True)
predicted_valid_1 = pd.Series(predicted_valid_1)
target_valid_1 = target_valid_1.reset_index(drop=True)
predicted_valid_2 = pd.Series(predicted_valid_2)
target_valid_2 = target_valid_2.reset_index(drop=True)

In [48]:
revenue_0 = (boot(target_valid_0, predicted_valid_0))

In [49]:
revenue_1 = (boot(target_valid_1, predicted_valid_1))

In [50]:
revenue_2 = (boot(target_valid_2, predicted_valid_2))

In [51]:
confidence_interval_0 = st.t.interval(0.95, len(revenue_0)-1, revenue_0.mean(), revenue_0.sem())

In [52]:
confidence_interval_1 = st.t.interval(0.95, len(revenue_1)-1, revenue_1.mean(), revenue_1.sem())

In [53]:
confidence_interval_2 = st.t.interval(0.95, len(revenue_2)-1, revenue_2.mean(), revenue_2.sem())

In [54]:
def risk_of_loss(revenue):
    count = 0
    for i in range(len(revenue)):
        if revenue[i] < budget:
            count += 1
    return count / 1000

Запишем все в таблицу:

In [55]:
d = pd.DataFrame(index=['geo_0', 'geo_1', 'geo_2'])
d['revenue_mean'] = [revenue_0.mean(), revenue_1.mean(), revenue_2.mean()]
d['confidence_interval'] = [confidence_interval_0, confidence_interval_1, confidence_interval_2]
d['risk_of_loss_%'] = [risk_of_loss(revenue_0) * 100, risk_of_loss(revenue_1) * 100, risk_of_loss(revenue_2) * 100] 
d

,revenue_mean,confidence_interval,risk_of_loss_%
geo_0,1.060074e+10,"(10580991087.318878, 10620479401.20345)",2.0
geo_1,1.066524e+10,"(10648855519.467796, 10681626596.97435)",0.3
geo_2,1.061556e+10,"(10595641274.024086, 10635478171.657852)",3.0


##### 5.3. Вывод

После оценки рисков оставим регионы, в которых вероятность убытков меньше 2.5%. Это geo_0 и geo_1. Для разработки скважин можно предложить регион geo_1, так как в нем получилась самая большая средняя прибыль.